In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def to_str(num):
  if not (type(num) == str):
    num = round(num)
    num = str (num)
  return num

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간랭킹/20210706일간랭킹_모든항목.csv')

#nan값 지우기
df = df.fillna('')

#if '태그' in df.columns:
df['태그'] = ''

#장르 리스트화
genres = ['발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'JPOP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']

# 번호 전부 문자열로
df['번호'] = df['번호'].apply(to_str)

#에러는 따로 모아놓자
err_list = []
for idx in range(len(df['번호'].values)):
  url = main_url + df['번호'].iloc[idx]
  time.sleep(3)
  driver.get(url)
  try:
    body = driver.find_element_by_css_selector('body')
    res = driver.page_source
    soup = bs(res, "html.parser")

    #장르 업데이트
    print(df['장르'].iloc[idx], '->', end='')
    if not (df['장르'].iloc[idx] in genres):
      dl_list = soup.find('dl', {'class':'list'})
      dl_list_text = dl_list.text
      dl_list_text = dl_list_text.split('\n')
      genre_index = dl_list_text.index('장르')
      df['장르'].iloc[idx] = dl_list_text[genre_index+1]

    #노래에 추천 태그 긁어오기
    div_service_list_album = soup.find('div', {'class':'service_list_album'})
    tag_ul = div_service_list_album.find('ul')
    tag_div = tag_ul.find_all('div', {'class':'tag_list'})
    tag_list= []

    #긁어온 태그 리스트화
    for div in tag_div:
      tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')

    #기존에 있던 태그 리스트화
    tag_list_ori = df['태그'].iloc[idx].split(' ')

    #기존태그 + 새로운태그 and set화하여 중복 제거 및 공백태그제거
    tag_list += tag_list_ori
    tag_set = set(tag_list)
    if '' in tag_set:
      tag_set.remove('')
    df['태그'].iloc[idx] = (' ').join(tag_set)
    
    #진행사항 표시
    print(df['장르'].iloc[idx], df['제목'].iloc[idx], df['태그'].iloc[idx], idx+1, '/', len(df['번호'].values))
  except:
    err_list.append(idx)
    continue
#파일로 저장

print('error num', len(err_list))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간랭킹/20210706일간랭킹_모든항목.csv')

#테스트

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간랭킹/20210706일간랭킹_모든항목.csv')

#nan값 지우기
df = df.fillna('')

#if '태그' in df.columns:
df['태그'] = ''

#장르 리스트화
genres = ['발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'JPOP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']

# 번호 전부 문자열로
df['번호'] = df['번호'].apply(to_str)

#에러는 따로 모아놓자
err_list = []
for idx in range(1):
  url = main_url + df['번호'].iloc[idx]
  time.sleep(3)
  driver.get(url)
  try:
    body = driver.find_element_by_css_selector('body')
    res = driver.page_source
    soup = bs(res, "html.parser")

    #장르 업데이트
    print(df['장르'].iloc[idx], '->', end='')
    if not (df['장르'].iloc[idx] in genres):
      dl_list = soup.find('dl', {'class':'list'})
      dl_list_text = dl_list.text
      dl_list_text = dl_list_text.split('\n')
      genre_index = dl_list_text.index('장르')
      df['장르'].iloc[idx] = dl_list_text[genre_index+1]

    #노래에 추천 태그 긁어오기
    div_service_list_album = soup.find('div', {'class':'service_list_album'})
    tag_ul = div_service_list_album.find('ul')
    tag_div = tag_ul.find_all('div', {'class':'tag_list'})
    tag_list= []

    #긁어온 태그 리스트화
    for div in tag_div:
      tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')

    #기존에 있던 태그 리스트화
    tag_list_ori = df['태그'].iloc[idx].split(' ')

    #기존태그 + 새로운태그 and set화하여 중복 제거 및 공백태그제거
    tag_list += tag_list_ori
    tag_set = set(tag_list)
    if '' in tag_set:
      tag_set.remove('')
    df['태그'].iloc[idx] = (' ').join(tag_set)
    
    #진행사항 표시
    print(df['장르'].iloc[idx], df['제목'].iloc[idx], df['태그'].iloc[idx], idx+1, '/', len(df['번호'].values))
  except:
    err_list.append(idx)
    continue

In [ ]:
df

,Unnamed: 0,날짜,번호,제목,가수,장르,앨범이름,좋아요,태그
0,0,20210706,33625988,바라만 본다,MSG워너비(M.O.M),발라드,MSG워너비 1집,"88,567",#기분전환 #트렌디 #발라드 #봄
1,0,20210706,33487342,Next Level,aespa,장르종합,Next Level,"154,848",
2,0,20210706,33625989,나를 아는 사람,MSG워너비(정상동기),장르종합,MSG워너비 1집,"56,333",
3,0,20210706,33507137,Butter,방탄소년단,장르종합,Butter,"216,789",
4,0,20210706,33503722,헤픈 우연,헤이즈 (Heize),장르종합,HAPPEN,"120,987",
...,...,...,...,...,...,...,...,...,...
2695,0,20210706,5718108,군밤타령,송소희,국악,송소희 얼쑤 아름다운 樂 콘서트,340,
2696,0,20210706,33013681,강강술래,서도밴드(sEODo BAND),국악,불후의 명곡 – 전설을 노래하다 (2020 국악인 특집),91,
2697,0,20210706,4318276,언제나 몇번이라도 (영화 '센과 치히로의 행방불명'),,국악,가야금 & 해금 베스트,"1,442",
2698,0,20210706,179780,어디로 갈꺼나,김영임,국악,‘96 김영임의 한(恨),274,


In [ ]:
df.isna().sum()

Unnamed: 0    0
날짜            0
번호            0
제목            0
가수            0
장르            0
앨범이름          0
좋아요           0
태그            0
dtype: int64